## ADRNX BB PID DEMO

### Initialization (make sure to shut down the magcycle kernel first!!!)

In [1]:
import serial, threading, sys, time, os, datetime, warnings, numpy as np, matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import FloatProgress, HTML, VBox
from timeit import default_timer as timer

# We will be using our homemade library to talk to ADR and BB via serial
sys.path.append('/home/pi/Desktop/ADRNX/SRS')
import SRS
sys.path.append('/home/pi/Desktop/ADRNX/BBNX')
import HCLib

In [2]:
# Initialize BB
HC = HCLib.HeaterController.Core(debug=True,override_device='/dev/ADR_hc')
HC.establish_comms()

HCDBG: Port ready: Serial<id=0x746435b0, open=False>(port=None, baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=0.2, xonxoff=False, rtscts=False, dsrdtr=False)
HCDBG: Platform (linux) -> device list is ['/dev/ADR_hc']
HCDBG: Starting device search
HCDBG: Port /dev/ADR_hc incorrect msg:   
HCDBG: Port /dev/ADR_hc incorrect msg: 
HCDBG: Port /dev/ADR_hc incorrect msg: I am HeaterController: 1E 95 F
HCDBG: Port /dev/ADR_hc incorrect msg: Type: A0
HCDBG: Port /dev/ADR_hc incorrect msg: FW: 12
HCDBG: Port /dev/ADR_hc incorrect msg: SN: 11
HCDBG: Port /dev/ADR_hc incorrect msg: Freq: 16000000
HCDBG: Port /dev/ADR_hc incorrect msg: Checking EEPROM...TYPE OK...SN OK!
HCDBG: Port /dev/ADR_hc incorrect msg: Timer 1 setup: TOP=31250 and f=4
HCDBG: Port /dev/ADR_hc CORRECT msg: HCA011
HCDBG: Found device on port /dev/ADR_hc -> connected!
HCDBG: Device READY!


In [3]:
# Initialize mainframe controller (with config file, suppressing debug statements)
mf = SRS.SIM900.SIM900('/home/pi/Desktop/ADRNX/SRS/config.yaml',debug=False)
mf.init_mainframe()
# Create module objects (coresponding to those in the rack) and add them to mainframe
s921 = SRS.SIM921.SIM921(mf,name='s921',debug=False)
s922 = SRS.SIM922.SIM922(mf,name='s922',debug=False)
s960 = SRS.SIM960.SIM960(mf,name='s960',debug=False)
s970 = SRS.SIM970.SIM970(mf,name='s970',debug=False)
mf.register_and_init_mods([s921, s922, s960, s970])
# Finally, start communication threads (this will enable all the backend virtual queues and messaging)
mf.start_comm_threads()

In [4]:
def make_bar(minv=0,maxv=1,prefix='Voltage: ',postfix='/2.0V',style='info'):
    progress = FloatProgress(min=minv,max=maxv,step=(maxv-minv)/1000)
    progress.bar_style = style
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    label.value = prefix+''+postfix
    return [progress, label, box,prefix,postfix]
    
def update_bar(bar,val,fmt='{:.3f}'):
    bar[0].value = val
    bar[1].value = bar[3]+fmt.format(val)+bar[4]
    if val ==  bar[0].max:  
        bar[0].bar_style = 'success' 

### Actual PID code

In [14]:
targetT = 20.0 #K
P = targetT*0.7
I = 0.03
D = 0.10
slewlimit = 0.5 #%/s

In [15]:
pid = HCLib.PID.stockPID(P,I,D,0)
pid.SetPoint = targetT
pid.setSlewLimit(slewlimit)

start = timer()
HC.set_dutycycle(0)
HC.set_onoff(1)
runPID = True

barBB = make_bar(0,targetT+5,prefix='BB temperature: ',postfix='K',style='warning')
barFAA = make_bar(0,500,prefix='FAA temperature: ',postfix='mK',style='info')
barDC = make_bar(0,100,prefix='Duty cycle: ',postfix='%',style='danger')

s921.start_data_stream()
s922.start_data_stream(4)
try:
    while runPID or timer()-start < 10:
        gotevent = s921.await_next_event(tm=2.5,clear_before=False,clear_after=True)
        if not gotevent: raise RuntimeError("No temperature in time - aborting")     
        tempFAA = s921.lastvalues[1]  
        
        gotevent = s922.await_next_event(tm=2.5,clear_before=True,clear_after=True)
        if not gotevent: raise RuntimeError("No temperature in time - aborting")            
        tempBB = s922.lastvalues[4]
        timeBB = s922.lastdatatimes[4] 
                
        # Override if too low or too high
        if targetT > tempBB+3.0:
            output = 100.0
        elif targetT < tempBB-0.2:
            output = 0.0
        else:
            pid.update(tempBB)
            output = pid.output
            
        # Clamp output     
        if output < 0.0:    
            output_actual = 0.0
        elif output > 100.0:
            output_actual = 100.0
        else:
            output_actual = output
            
        update_bar(barDC,output_actual)
        update_bar(barBB,tempBB)
        update_bar(barFAA,tempFAA*1000.0)
        
        HC.set_dutycycle(output_actual)               
        print("{:6.4f}|{:6.4f}|{:6.4f}|{:6.4f}".format(output_actual,output,tempBB,tempFAA))
except Exception as e:
    print(e)
    raise e
finally:
    s922.stop_data_stream()
    s921.stop_data_stream()
    HC.set_dutycycle(0)
    HC.set_onoff(0)

Sending command: |b'SC0000\n'|
Sending command: |b'ON0000\n'|


Sending command: |b'SC9999\n'|
100.0000|100.0000|15.7360|2.8461
Sending command: |b'SC9999\n'|
100.0000|100.0000|15.7330|2.8282
Sending command: |b'SC9999\n'|
100.0000|100.0000|15.7310|2.8034
Sending command: |b'SC9999\n'|
100.0000|100.0000|15.7460|2.8606
Sending command: |b'SC9999\n'|
100.0000|100.0000|15.7940|2.8587
Sending command: |b'SC9999\n'|
100.0000|100.0000|15.8670|2.8841
Sending command: |b'SC9999\n'|
100.0000|100.0000|15.9590|2.8829
Sending command: |b'SC9999\n'|
100.0000|100.0000|16.0660|2.8754
Sending command: |b'SC9999\n'|
100.0000|100.0000|16.1820|2.8861
Sending command: |b'SC9999\n'|
100.0000|100.0000|16.4320|2.8443
Sending command: |b'SC9999\n'|
100.0000|100.0000|16.5630|2.8644
Sending command: |b'SC9999\n'|
100.0000|100.0000|16.6970|2.8609
Sending command: |b'SC9999\n'|
100.0000|100.0000|16.8310|2.8724
Sending command: |b'SC0590\n'|
5.9055|5.9055|17.0990|2.8764
Sending command: |b'SC0630\n'|
6.3009|6.3009|17.2320|2.8885
Sending command: |b'SC0670\n'|
6.7050|6.7050|17.

/home/pi/Desktop/ADRNX/SRS/SIM922.py:122: UserWarning: SIM922 BAD TEMP LENGTH: |'1.94'|
  warnings.warn("SIM922 BAD TEMP LENGTH: |{}|".format(repr(val)))
/home/pi/Desktop/ADRNX/SRS/SIM922.py:122: UserWarning: SIM922 BAD TEMP LENGTH: |'0800E+01'|
  warnings.warn("SIM922 BAD TEMP LENGTH: |{}|".format(repr(val)))


Sending command: |b'SC1261\n'|
12.6152|12.6152|19.4230|2.8431
Sending command: |b'SC1249\n'|
12.4982|12.4982|19.4370|2.8344
Sending command: |b'SC1237\n'|
12.3803|12.3803|19.4510|2.7997
Sending command: |b'SC1227\n'|
12.2712|12.2712|19.4640|2.8352
Sending command: |b'SC1204\n'|
12.0422|12.0422|19.4910|2.8570
Sending command: |b'SC1193\n'|
11.9318|11.9318|19.5040|2.8838
Sending command: |b'SC1182\n'|
11.8213|11.8213|19.5170|2.8689
Sending command: |b'SC1171\n'|
11.7205|11.7205|19.5290|2.8491
Sending command: |b'SC1150\n'|
11.5058|11.5058|19.5540|2.8737
Sending command: |b'SC1140\n'|
11.4036|11.4036|19.5660|2.8982
Sending command: |b'SC1129\n'|
11.3003|11.3003|19.5780|2.9127
Sending command: |b'SC1119\n'|
11.1965|11.1965|19.5900|2.9170
Sending command: |b'SC1099\n'|
10.9979|10.9979|19.6130|2.9070
Sending command: |b'SC1090\n'|
10.9028|10.9028|19.6240|2.8599
Sending command: |b'SC1080\n'|
10.8071|10.8071|19.6350|2.8518
Sending command: |b'SC1071\n'|
10.7119|10.7119|19.6460|2.8587
Sending 

Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1100|2.8303
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1090|2.8459
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1080|2.8532
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1060|2.8708
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1020|2.8598
Sending command: |b'SC0603\n'|
6.0338|6.0338|20.1000|2.8344
Sending command: |b'SC0605\n'|
6.0600|6.0600|20.0970|2.8193
Sending command: |b'SC0608\n'|
6.0863|6.0863|20.0940|2.8406
Sending command: |b'SC0612\n'|
6.1290|6.1290|20.0890|2.8459
Sending command: |b'SC0613\n'|
6.1352|6.1352|20.0880|2.8079
Sending command: |b'SC0615\n'|
6.1520|6.1520|20.0860|2.8175
Sending command: |b'SC0615\n'|
6.1585|6.1585|20.0850|2.8050
Sending command: |b'SC0615\n'|
6.1515|6.1515|20.0850|2.7756
Sending command: |b'SC0614\n'|
6.1480|6.1480|20.0850|2.7456
Sending command: |b'SC0613\n'|
6.1345|6.1345|20.0860|2.7771
Sending command: |b'SC0613\n'|
6.1312|6.1312|20.0860|2.8003
Sending command: |b'SC0611\n'|
6.1141|6.

/home/pi/Desktop/ADRNX/SRS/SIM922.py:122: UserWarning: SIM922 BAD TEMP LENGTH: |'2.00'|
  warnings.warn("SIM922 BAD TEMP LENGTH: |{}|".format(repr(val)))
/home/pi/Desktop/ADRNX/SRS/SIM922.py:122: UserWarning: SIM922 BAD TEMP LENGTH: |'8800E+01'|
  warnings.warn("SIM922 BAD TEMP LENGTH: |{}|".format(repr(val)))


Sending command: |b'SC0608\n'|
6.0868|6.0868|20.0890|2.8500
Sending command: |b'SC0607\n'|
6.0733|6.0733|20.0900|2.8356
Sending command: |b'SC0606\n'|
6.0699|6.0699|20.0900|2.8261
Sending command: |b'SC0604\n'|
6.0458|6.0458|20.0920|2.8003
Sending command: |b'SC0601\n'|
6.0118|6.0118|20.0950|2.7793
Sending command: |b'SC0597\n'|
5.9779|5.9779|20.0980|2.8227
Sending command: |b'SC0596\n'|
5.9643|5.9643|20.0990|2.8315
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1050|2.8895
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1080|2.8774
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1100|2.8679
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1110|2.8887
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1130|2.8362
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1130|2.8476
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1120|2.8536
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1110|2.8505
Sending command: |b'SC0000\n'|
0.0000|0.0000|20.1080|2.8490
Sending command: |b'SC0000\n'|
0.0000|0.

KeyboardInterrupt: 

Code below is still in construction, dont use

In [ ]:
def updateSetpoint(b):
    targetT = ftext.value
    pid.SetPoint = targetT
    print(temp)
    

ftext = widgets.BoundedFloatText(
            value=7.5,
            min=0.0,
            max=25.0,
            step=0.1,
            description='Temperature setpoint:',
            disabled=False
        )
html = widgets.HTML(
    value="..........................",
    placeholder="..........................",
)

button = widgets.Button(
    description='Update temperature',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
)
button.on_click(updateSetpoint)
l1 = widgets.Label("0.333")
l2 = widgets.HTML(value="<center>   Hello</center>",)
hb1 = widgets.HBox([ftext,button])
fp = widgets.FloatProgress(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='FAA',
    bar_style='info',
    orientation='vertical'
)
fpfull = widgets.VBox([l2,fp])
fp3 = widgets.HBox([fpfull,fpfull,fpfull])
box = widgets.VBox([hb1,html,fp3])
display(box)

In [ ]:
for i in range(4):
    time.sleep(1)
    fp.description=str(i)